In [27]:
import time
import random
import sys
import os

### Інструменти для метрик

In [28]:
class SortMetrics:
    def __init__(self):
        self.comparisons = 0  # Кількість порівнянь
        self.copies = 0       # Кількість копіювань (присвоєнь)
        self.start_time = 0
        self.end_time = 0

    def start(self):
        self.comparisons = 0
        self.copies = 0
        self.start_time = time.time()

    def stop(self):
        self.end_time = time.time()

    def add_compare(self):
        self.comparisons += 1

    def add_copy(self):
        self.copies += 1

    def get_time(self):
        return self.end_time - self.start_time

In [29]:
# Обгортка для елементів масиву, щоб автоматично рахувати порівняння
class TrackedItem:
    def __init__(self, value, metrics):
        self.value = value
        self.metrics = metrics

    def __lt__(self, other):
        self.metrics.add_compare()
        return self.value < other.value

    def __le__(self, other):
        self.metrics.add_compare()
        return self.value <= other.value

    def __repr__(self):
        return repr(self.value)

### Реалізація алгоритмів

In [30]:
# А) Рекурсивне сортування злиттям (Top-Down) 
def merge_sort_recursive(arr, metrics):
    if len(arr) <= 1:
        return arr
    
    mid = len(arr) // 2
    left = merge_sort_recursive(arr[:mid], metrics)
    right = merge_sort_recursive(arr[mid:], metrics)
    
    return merge(left, right, metrics)

def merge(left, right, metrics):
    sorted_arr = []
    i = j = 0
    while i < len(left) and j < len(right):
        if left[i] <= right[j]:
            sorted_arr.append(left[i])
            metrics.add_copy()
            i += 1
        else:
            sorted_arr.append(right[j])
            metrics.add_copy()
            j += 1
    
    while i < len(left):
        sorted_arr.append(left[i])
        metrics.add_copy()
        i += 1
    while j < len(right):
        sorted_arr.append(right[j])
        metrics.add_copy()
        j += 1
    return sorted_arr


In [31]:
# Б) Ітеративне сортування злиттям (Bottom-Up) 
def merge_sort_iterative(arr, metrics):
    width = 1
    n = len(arr)
    temp_arr = arr[:] # Копія масиву
    
    while width < n:
        for i in range(0, n, 2 * width):
            left = i
            mid = min(i + width, n)
            right = min(i + 2 * width, n)
            
            # Злиття підмасивів
            l, r = left, mid
            k = left
            while l < mid and r < right:
                if temp_arr[l] <= temp_arr[r]:
                    arr[k] = temp_arr[l]
                    l += 1
                else:
                    arr[k] = temp_arr[r]
                    r += 1
                metrics.add_copy()
                k += 1
            
            while l < mid:
                arr[k] = temp_arr[l]
                metrics.add_copy()
                l += 1
                k += 1
            while r < right:
                arr[k] = temp_arr[r]
                metrics.add_copy()
                r += 1
                k += 1
                
        temp_arr = arr[:] # Оновлюємо стан
        width *= 2
    return arr

In [ ]:
# В) Оптимізоване сортування злиттям 
# 1. Cutoff to insertion sort
# 2. Stop-if-already-sorted
# 3. Eliminate copy

def insertion_sort(arr, l, r, metrics):
    for i in range(l + 1, r + 1):
        key = arr[i]
        j = i - 1
        metrics.add_copy()
        while j >= l and arr[j] > key:
            arr[j + 1] = arr[j]
            metrics.add_copy()
            j -= 1
        arr[j + 1] = key
        metrics.add_copy()

def merge_optimized(src, dst, l, mid, r, metrics):
    # Оптимізація: Stop-if-already-sorted
    if src[mid] <= src[mid+1]:
        for i in range(l, r + 1):
            dst[i] = src[i]
            metrics.add_copy()
        return

    i, j, k = l, mid + 1, l
    while i <= mid and j <= r:
        if src[i] <= src[j]:
            dst[k] = src[i]
            i += 1
        else:
            dst[k] = src[j]
            j += 1
        metrics.add_copy()
        k += 1
    
    while i <= mid:
        dst[k] = src[i]
        metrics.add_copy()
        k += 1
        i += 1
    while j <= r:
        dst[k] = src[j]
        metrics.add_copy()
        k += 1
        j += 1

def sort_opt_recursive(src, dst, l, r, metrics):
    # Оптимізація: Cutoff to insertion (для масивів <= 10 елементів)
    if r - l <= 10: 
        insertion_sort(dst, l, r, metrics)
        return

    mid = l + (r - l) // 2
    # Оптимізація: Eliminate copy (міняємо місцями src і dst у рекурсії)
    sort_opt_recursive(dst, src, l, mid, metrics)
    sort_opt_recursive(dst, src, mid + 1, r, metrics)
    merge_optimized(src, dst, l, mid, r, metrics)

def merge_sort_optimized_entry(arr, metrics):
    # Початковий допоміжний масив створюється лише 1 раз
    sort_opt_recursive(arr[:], arr, 0, len(arr) - 1, metrics)
    return arr

In [33]:
# Г) Сортирування зв'язного списку (Linked List) 
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

def sorted_merge_list(a, b, metrics):
    result = None
    if a is None: return b
    if b is None: return a

    if a.val <= b.val:
        result = a
        result.next = sorted_merge_list(a.next, b, metrics)
    else:
        result = b
        result.next = sorted_merge_list(a, b.next, metrics)
    metrics.add_copy() # Рахуємо перелінковку як операцію
    return result

def merge_sort_linked_list(head, metrics):
    if head is None or head.next is None:
        return head

    # Знаходимо середину списку
    middle = get_middle(head)
    next_to_middle = middle.next
    middle.next = None

    left = merge_sort_linked_list(head, metrics)
    right = merge_sort_linked_list(next_to_middle, metrics)

    return sorted_merge_list(left, right, metrics)

def get_middle(head):
    if head is None: return head
    slow = head
    fast = head
    while fast.next is not None and fast.next.next is not None:
        slow = slow.next
        fast = fast.next.next
    return slow

def run_linked_list_sort(arr, metrics):
    # Конвертація масиву в список
    if not arr: return []
    head = ListNode(arr[0])
    curr = head
    for x in arr[1:]:
        curr.next = ListNode(x)
        curr = curr.next
    
    # Сортування
    new_head = merge_sort_linked_list(head, metrics)
    
    # Конвертація назад у масив (для перевірки)
    res = []
    while new_head:
        res.append(new_head.val)
        new_head = new_head.next
    return res

### Генерація даних

In [34]:
def generate_data(size, data_type="random"):
    if data_type == "random": # Випадкові
        return [random.randint(0, size*10) for _ in range(size)]
    elif data_type == "sorted": # Відсортовані
        return list(range(size))
    elif data_type == "reversed": # Зворотні
        return list(range(size, 0, -1))
    elif data_type == "nearly_sorted": # Майже відсортовані
        arr = list(range(size))
        # Міняємо 5% елементів місцями
        for _ in range(size // 20):
            i, j = random.randint(0, size-1), random.randint(0, size-1)
            arr[i], arr[j] = arr[j], arr[i]
        return arr
    elif data_type == "few_unique": # Мало унікальних значень
        return [random.randint(0, 10) for _ in range(size)]
    return []

### Тестування та аналіз

In [ ]:
def run_benchmark():
    sizes = [1000, 5000] # Розміри масивів
    types = ["random", "sorted", "reversed", "nearly_sorted", "few_unique"]
    
    print(f"\n{'Розмір':<10} | {'Тип даних':<15} | {'Алгоритм':<18} | {'Час (с)':<10} | {'Порівнянь':<10} | {'Копіювань':<10}")
    print("-" * 90)

    for size in sizes:
        for dtype in types:
            raw_data = generate_data(size, dtype)
            
            algorithms = [
                ("Рекурсивний", merge_sort_recursive),
                ("Ітеративний", merge_sort_iterative),
                ("Оптимізований", merge_sort_optimized_entry),
                ("Зв'язний список", run_linked_list_sort)
            ]

            for name, func in algorithms:
                metrics = SortMetrics()
                # Обгортаємо дані для підрахунку порівнянь
                test_data = [TrackedItem(x, metrics) for x in raw_data]
                
                try:
                    metrics.start()
                    func(test_data, metrics)
                    metrics.stop()
                    print(f"{size:<10} | {dtype:<15} | {name:<18} | {metrics.get_time():<10.5f} | {metrics.comparisons:<10} | {metrics.copies:<10}")
                except RecursionError:
                    print(f"{size:<10} | {dtype:<15} | {name:<18} | {'ПОМИЛКА':<10} | {'RECURSION':<10} | {'-'}")

In [36]:
run_benchmark()


Розмір     | Тип даних       | Алгоритм           | Час (с)    | Порівнянь  | Копіювань 
------------------------------------------------------------------------------------------
1000       | random          | Рекурсивний        | 0.04081    | 8675       | 9976      
1000       | random          | Ітеративний        | 0.03679    | 8702       | 10000     
1000       | random          | Оптимізований      | 0.03427    | 9215       | 10433     
1000       | random          | Зв'язний список    | 0.03748    | 8677       | 8677      
1000       | sorted          | Рекурсивний        | 0.02849    | 4932       | 9976      
1000       | sorted          | Ітеративний        | 0.02698    | 5052       | 10000     
1000       | sorted          | Оптимізований      | 0.01345    | 999        | 8744      
1000       | sorted          | Зв'язний список    | 0.02427    | 5044       | 5044      
1000       | reversed        | Рекурсивний        | 0.02775    | 5044       | 9976      
1000       | rever

# Звіт

### 1. Мета роботи
Реалізація різних модифікацій алгоритму сортування злиттям (Merge Sort) та проведення порівняльного аналізу їх ефективності за часом виконання, кількістю порівнянь та кількістю операцій копіювання на масивах різного типу та розміру.

### 2. Результати вимірювань
В ході роботи було протестовано 4 реалізації алгоритму:
1.  **Рекурсивний (Recursive)** — класична реалізація Top-Down.
2.  **Ітеративний (Iterative)** — реалізація Bottom-Up (без рекурсії).
3.  **Оптимізований (Optimized)** — включає перевірку на відсортованість, відсікання на сортування вставками (Insertion Sort) для малих підмасивів та зменшення копіювань.
4.  **Зв'язний список (Linked List)** — сортування елементів, організованих у зв'язний список.

### 3. Аналіз результатів

На основі отриманих даних можна зробити наступні висновки:

**А. Порівняння Рекурсивного та Ітеративного методів**
На випадкових даних (random) ітеративний метод показує трохи кращу швидкодію (наприклад, для N=5000: 0.117с проти 0.132с у рекурсивного). Це пояснюється відсутністю накладних витрат на виклик функцій (стек викликів), які присутні у рекурсивному варіанті. За кількістю порівнянь та копіювань вони майже ідентичні, що підтверджує однакову асимптотичну складність $O(N \log N)$.

**Б. Ефективність оптимізацій**
Оптимізований алгоритм продемонстрував найкращі результати на специфічних наборах даних:
* **Відсортовані дані (Sorted):** Найбільший приріст продуктивності. Для N=5000 час склав 0.038с проти 0.091с у звичайного алгоритму. Це досягається завдяки перевірці `if (arr[mid] <= arr[mid+1]) return;`, що дозволяє пропускати етап злиття. Кількість порівнянь при цьому мінімальна (4999 для 5000 елементів, тобто $N-1$).
* **Майже відсортовані (Nearly Sorted):** Оптимізація також суттєва (0.078с проти 0.121с для N=5000), оскільки багато підмасивів вже впорядковані.
* **Зворотний порядок (Reversed):** Оптимізований метод показав гірші результати за кількістю копіювань, що може бути пов'язано з витратами на спроби застосувати Insertion Sort до даних, які є "найгіршим випадком" для вставок.

**В. Сортування зв'язного списку**
Реалізація на зв'язному списку (Linked List) виявилася найповільнішою на великих обсягах даних (0.160с для N=5000 Random), незважаючи на те, що кількість "копіювань" (змін вказівників) менша або дорівнює масивам. Це зумовлено поганою локальністю кешу (елементи списку розкидані в пам'яті), тоді як масиви займають неперервну область пам'яті, що прискорює доступ процесора до даних.

### 4. Висновки
1.  Алгоритм сортування злиттям є стабільним та гарантує час виконання $O(N \log N)$ для будь-яких вхідних даних.
2.  **Ітеративна реалізація** є дещо швидшою за рекурсивну на практиці через відсутність витрат на системний стек.
3.  **Оптимізації** (Stop-if-sorted, перехід на вставки) є критично важливими для реальних завдань, де дані часто можуть бути частково впорядкованими. У найкращому випадку (вже відсортований масив) складність наближається до $O(N)$.
4.  Для роботи з великими обсягами даних у пам'яті масиви є кращими за зв'язні списки через архітектурні особливості сучасних процесорів (кешування), навіть якщо алгоритмічна складність однакова.